# Learn Posture

use machine learning to recognize robot's posture (following the example in [scikit-learn-intro.ipynb](./scikit-learn-intro.ipynb) )

## 1. Data collection

We have colleceted data before, you need to add new data if you want to add new posture.

* the dateset are in *robot_pose_data* folder
* each file contains the data belongs to this posture, e.g. the data in *Back* file are collected when robot was in "Back" posture
* the data file can be load by ```pickle```, e.g. ```pickle.load(open('Back'))```, the data is a list of feature data
* the features (e.g. each row of the data) are ['LHipYawPitch', 'LHipRoll', 'LHipPitch', 'LKneePitch', 'RHipYawPitch', 'RHipRoll', 'RHipPitch', 'RKneePitch', 'AngleX', 'AngleY'], where 'AngleX' and 'AngleY' are body angle (e.g. ```Perception.imu```) and others are joint angles.

## 2. Data preprocessing

In [1]:
%pylab inline
import pickle
from os import listdir, path
import numpy as np
from sklearn import svm, metrics

ROBOT_POSE_DATA_DIR = 'robot_pose_data'

Populating the interactive namespace from numpy and matplotlib


In [2]:
classes = listdir(ROBOT_POSE_DATA_DIR)
print(classes)

['Sit', 'Crouch', 'Belly', 'StandInit', 'Back', 'Right', 'Left', 'Knee', 'HeadBack', 'Frog', 'Stand']


In [4]:
def load_pose_data(i):
    '''load pose data from file'''
    data = []
    target = []
    # YOUR CODE HERE
    # TODO: what is left to do here???
        
    filename = path.join(ROBOT_POSE_DATA_DIR, classes[i])
    # print(filename)
    data = pickle.load(open(filename, "rb"))
    target = [i] * len(data)
    return data, target

In [8]:
# load all the data
all_data = []
all_target = []
# YOUR CODE HERE
for i,_ in enumerate(classes):
    d, t = load_pose_data(i)
    all_data.extend(d)
    all_target.extend(t)
    
# all_data = np.array(all_data)
# all_target = np.array(all_target)
print('total number of data', len(all_data))
all_data = np.asarray(all_data)
all_target = np.asarray(all_target)
# print(all_data)

[[-0.7562201023101807, 0.7839159965515137, -1.630600094795227, 0.2607381343841553, -0.7562201023101807, 0.09975194931030273, -1.5708580017089844, 0.1074218824505806, -0.03839683532714844, 0.438077449798584], [-0.7562201023101807, 0.7839159965515137, -1.630600094795227, 0.2607381343841553, -0.7562201023101807, 0.09975194931030273, -1.5708580017089844, 0.1074218824505806, -0.036651611328125, 0.438077449798584], [-0.7562201023101807, 0.7839159965515137, -1.630600094795227, 0.2607381343841553, -0.7562201023101807, 0.09821796417236328, -1.5708580017089844, 0.1074218824505806, -0.036651611328125, 0.43633222579956055], [-0.7562201023101807, 0.7823820114135742, -1.630600094795227, 0.2607381343841553, -0.7562201023101807, 0.09975194931030273, -1.5708580017089844, 0.1074218824505806, -0.03839683532714844, 0.44156789779663086], [-0.7562201023101807, 0.7839159965515137, -1.630600094795227, 0.2607381343841553, -0.7562201023101807, 0.09975194931030273, -1.5708580017089844, 0.1074218824505806, -0.038

In [16]:
# shuffule data
permutation = np.random.permutation(len(all_data))
n_training_data = int(len(all_data) * 0.7)
training_data = permutation[:n_training_data]

## 3. Learn on training data

In scikit-learn, an estimator for classification is a Python object that implements the methods fit(X, y) and predict(T). An example of an estimator is the class sklearn.svm.SVC that implements support vector classification.

In [17]:
clf = svm.SVC(gamma=0.001, C=100.)

### learning

In [19]:
# YOUR CODE HERE
clf.fit(all_data[training_data], all_target[training_data])


SVC(C=100.0, gamma=0.001)

### predicting

In [22]:
# original is not working:
# clf.predict(all_data[-1]), all_target[-1]

# this is
clf.predict(all_data[-1:]), all_target[-1:]

(array([10]), array([10]))

In [23]:
def evaluate(expected, predicted):
    print("Classification report:\n%s\n" % metrics.classification_report(expected, predicted))

    print("Confusion matrix:\n%s" % metrics.confusion_matrix(expected, predicted))

In [24]:
expected = []
predicted = []
# YOUR CODE HERE
expected = np.asarray(all_target)[training_data]
predicted = clf.predict(np.asarray(all_data)[training_data])

evaluate(expected, predicted)

Classification report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        21
           1       0.96      1.00      0.98        25
           2       1.00      1.00      1.00        13
           3       1.00      1.00      1.00        33
           4       1.00      1.00      1.00        16
           5       1.00      0.86      0.92         7
           6       1.00      1.00      1.00        15
           7       1.00      1.00      1.00         8
           8       1.00      1.00      1.00         4
           9       1.00      1.00      1.00         8
          10       1.00      1.00      1.00         6

    accuracy                           0.99       156
   macro avg       1.00      0.99      0.99       156
weighted avg       0.99      0.99      0.99       156


Confusion matrix:
[[21  0  0  0  0  0  0  0  0  0  0]
 [ 0 25  0  0  0  0  0  0  0  0  0]
 [ 0  0 13  0  0  0  0  0  0  0  0]
 [ 0  0  0 33  0  0  0  0  0  0  0]
 

## 4. Evaluate on the test data

In [31]:
expected = []
predicted = []
# YOUR CODE HERE
testing_data = permutation[n_training_data:]
expected = np.asarray(all_target)[testing_data]
predicted = clf.predict(np.asarray(all_data)[testing_data])

# evaluate(expected, predicted)
len(testing_data) + len(training_data)

224

## 5. Deploy to the real system

We can simple use `pickle` module to serialize the trained classifier.

In [32]:
import pickle
ROBOT_POSE_CLF = 'robot_pose.pkl'
pickle.dump(clf, open(ROBOT_POSE_CLF, 'wb')) # here I added wb to the open call

Then, in the application we can load the trained classifier again.

In [33]:
clf2 = pickle.load(open(ROBOT_POSE_CLF, 'rb')) # I added rb to the open call
clf2.predict(all_data[-1:]), all_target[-1:]

(array([10]), array([10]))